# levenshtein analyse
Via de code in dit bestand heb ik de colommen geanalyseerd. De levenshtein distance is een metriek om de minimale bewerkafstand tussen twee strings te kunnen analyseren waaronder verwijdering, verandering of toevoeging van tekens. Omdat dit een intensieve bewerking is heb ik dit niet zelf gecode maar het via de Levenshtein libary geimplementeerd om zo tot performance verbetering te komen. Het idee is dat als er beperkt aantal foutjes zijn (3 of minder) dan kan je dit detecteren in een uitdraai van de levenshtein-distance

In [3]:
from genlib import get_unique_values, get_csv
import pandas as pd
from Levenshtein import distance

Als eerste heb ik de methode getest om te weten of hij werkt zoals ik verwacht

In [ ]:
def calc_distance(str1:str, str2:str)->int:
	return distance(str1, str2)

a = calc_distance("ab", "a") #1 expected
b = calc_distance("ab", "aa") #1 expected
c = calc_distance("aabbcc", "abc") #3 expected
d = calc_distance("ab", "abcabc") #4 expected

print(f"{a}\n{b}\n{c}\n{d}")

1
1
3
4


Verder wou ik de subject colomn testen op mogelijke kleine foutjes dus heb ik de volgende code gemaakt

In [5]:
def levenstein_distance_column(df:pd.DataFrame, col_name:str, distance_limit:int=3):
	out = {}
	names_df = get_unique_values(df, col_name)
	df_sorted = names_df.sort_values(by='count', ascending=False).reset_index(drop=True)
	df_sorted2 = df_sorted.copy()

	index = 0
	for item in df_sorted.itertuples():
		df_sorted2.drop(index, inplace=True)
		att = getattr(item, col_name)
		
		for item2 in df_sorted2.itertuples():
			att2 = getattr(item2, col_name)

			dist = distance(att, att2)
			if dist <= distance_limit:
				if dist not in out:
					out[dist] = []
				out[dist].append(f"{att} -> {att2}")
		index+=1
	return out

def to_string_levenstein_distance_column(levenstein:dict)->str:
	out = ""

	for key in levenstein:
		out += f"distance[{key}]\n"
		for item in levenstein[key]:
			out += f"\t{item}\n"
		out += "\n"
	return out

Hieronder de print van de colomn verschillen tot 3 gesorteerd op groot naar klein.

In [6]:
df = get_csv('data/source.csv', 'Record ID', ['Record ID', 'Subject', 'Country', 'ArticleType', 'Reason' ])
dic = levenstein_distance_column(df, "Subject")
out = to_string_levenstein_distance_column(dic)
print(out)

distance[3]
	(HSC) Medicine - Pediatrics -> (HSC) Medicine - Geriatric
	(BLS) Nutrition -> (HSC) Nutrition
	(SOC) Philosophy -> (HUM) Philosophy
	(HUM) History - Asia -> (HUM) History - Africa
	(BLS) Forensic Sciences -> (PHY) Forensic Sciences

distance[2]
	(HUM) History - North America -> (HUM) History - South America




Daarna heb ik de onderstaande code geschreven om de levenshtein distance op meerdere colommen tegelijk te kunnen toepassen

In [ ]:
def levenstein_distance_dataframe(df:pd.DataFrame, distance_limit:int=3):
	out = {}

	col_names = df.columns
	for c_name in col_names:
		out[c_name] = levenstein_distance_column(df, c_name, distance_limit=distance_limit)

	return out

def to_string_levenstein_distance_dataframe(levenstein:dict)->str:
	out = ""

	for column_key in levenstein:
		out += f"{column_key}\n"

		column = levenstein[column_key]
		for distance in column:
			out += f"\tdistance[{distance}]\n"
			for item in column[distance]:
				out += f"\t\t{item}\n"
			out += "\n"
		out += "\n"
	return out

Als laatste heb ik de colommen: Subject, Country, ArticleType en Reason geanalyseerd met de gemaakte code. Andere colommen heb ik niet geanalyseerd omdat de get_unique_values() daar niet mee omkan gaan en mijn levenshtein functies daarvan afhankelijk van gemaakt heb.

In de uitslag eronder zie je dat Philosophy en Nutrition 2 keer voorkomen onder een andere hoofdcategorie. Het is de vraag of dit de bedoeling was.
Deze vind je onder Subject>distance[3]>...

In [8]:
df = get_csv('data/source.csv', 'Record ID', ['Record ID', 'Subject', 'Country', 'ArticleType', 'Reason' ])
dic = levenstein_distance_dataframe(df)
out = to_string_levenstein_distance_dataframe(dic)
print(out)

Subject
	distance[3]
		(HSC) Medicine - Pediatrics -> (HSC) Medicine - Geriatric
		(BLS) Nutrition -> (HSC) Nutrition
		(SOC) Philosophy -> (HUM) Philosophy
		(HUM) History - Asia -> (HUM) History - Africa
		(BLS) Forensic Sciences -> (PHY) Forensic Sciences

	distance[2]
		(HUM) History - North America -> (HUM) History - South America


Country
	distance[2]
		China -> Ghana
		China -> Chile
		Iran -> Oman
		South Korea -> North Korea
		Canada -> Panama
		Australia -> Austria
		Romania -> Somalia
		Nigeria -> Algeria
		Nigeria -> Niger
		Nigeria -> Liberia
		Serbia -> Syria
		Malta -> Mali
		Ghana -> Guyana
		Slovakia -> Slovenia
		Zambia -> Namibia
		Rwanda -> Uganda
		Gambia -> Namibia
		Aruba -> Cuba
		Mali -> Malawi

	distance[3]
		China -> Cuba
		China -> Chad
		India -> Syria
		Russia -> Tunisia
		Russia -> Austria
		Iran -> Japan
		Iran -> Italy
		Iran -> France
		Iran -> Spain
		Iran -> Jordan
		Iran -> Israel
		Iran -> Ireland
		Iran -> Ghana
		Iran -> Sudan
		Iran -> Chad
		J